In [ ]:
from google.colab import drive
drive.mount('/content/drive') # , force_remount=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd 
import numpy as np
import re
import os
import ast
from glob import glob
from tqdm import tqdm

import ast
tqdm.pandas()
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 200)

In [ ]:
def flatten_list(l):
  return [item for sublist in l for item in sublist]

from collections import Counter

In [ ]:
metadata_df = pd.read_csv('/content/drive/MyDrive/metavis/socrata_nycopendata_metadata_fetched_20211105.csv')
metadata_df['columns'] = metadata_df['columns'].apply(ast.literal_eval)

In [ ]:
# metadata_df['excel_attachments'] = metadata_df['data_uuid'].progress_apply(lambda data_uuid: sorted(glob('/content/drive/MyDrive/metavis/data_attachments/'+data_uuid+'/*.xlsx')))

# def get_excel_header_and_first_row(excel_attachment_path, sheet_number):
#   try:
#     excel_df = pd.read_excel(excel_attachment_path, sheet_name=sheet_number, nrows=0)
#     header = excel_df.columns.tolist()
#     excel_df = pd.read_excel(excel_attachment_path, sheet_name=sheet_number, nrows=0, header = 1)
#     first_row = excel_df.columns.tolist()
#     return header, first_row
#   except:
#     return [], []

# for sheet_number in [1,2]:
#   print(sheet_number)
#   metadata_df['header_and_first_row_of_excel_files_sheet'+str(sheet_number)] = metadata_df['excel_attachments'].progress_apply(lambda li: [get_excel_header_and_first_row(p, sheet_number = sheet_number) for p in li])
#   metadata_df['excel_header_signature_sheet'+str(sheet_number)] = metadata_df['header_and_first_row_of_excel_files_sheet'+str(sheet_number)].progress_apply(lambda files: [ '||'.join(['|'.join([str(field) for field in row[:3]]) for row in file]) for file in files] ) 
#   metadata_df['attachment_is_data_dictionary_sheet'+str(sheet_number)] = metadata_df['excel_header_signature_sheet'+str(sheet_number)].apply(lambda li: ['Data Dictionary - Column Information' in sig or 'Column Name|Column Description' in sig or 'Column Name|Description' in sig for sig in li])

# metadata_df['data_dictionary_file_indices_in_excel_attachments'] = metadata_df[['excel_attachments','attachment_is_data_dictionary_sheet1','attachment_is_data_dictionary_sheet2']].apply(lambda row: list(np.where(np.int0(np.array(row['attachment_is_data_dictionary_sheet1'])|np.array(row['attachment_is_data_dictionary_sheet2']))==1)[0]) if len(row['excel_attachments'])>0 else [], axis=1)
# # metadata_df[(metadata_df.excel_attachments.apply(len)>0) & (metadata_df.data_dictionary_file_indices_in_excel_attachments.apply(len)==0)]
# metadata_df['data_dictionary_file_path'] = metadata_df[['excel_attachments','data_dictionary_file_indices_in_excel_attachments']].apply(lambda row: row['excel_attachments'][row['data_dictionary_file_indices_in_excel_attachments'][0]] if len(row['data_dictionary_file_indices_in_excel_attachments'])>0 else np.nan,axis=1)
# # metadata_df[(metadata_df['columns'].apply(len)==0) & (metadata_df['data_dictionary_file_path'].notnull())]
# metadata_df['sheet1_is_data_dictionary'] = metadata_df[['attachment_is_data_dictionary_sheet1','data_dictionary_file_indices_in_excel_attachments']].apply(lambda row: row['attachment_is_data_dictionary_sheet1'][row['data_dictionary_file_indices_in_excel_attachments'][0]] if len(row['data_dictionary_file_indices_in_excel_attachments'])>0 else np.nan,axis=1)
# metadata_df['sheet2_is_data_dictionary'] = metadata_df[['attachment_is_data_dictionary_sheet2','data_dictionary_file_indices_in_excel_attachments']].apply(lambda row: row['attachment_is_data_dictionary_sheet2'][row['data_dictionary_file_indices_in_excel_attachments'][0]] if len(row['data_dictionary_file_indices_in_excel_attachments'])>0 else np.nan,axis=1)

# def get_data_dictionary(path, sheet_num, data_uuid):
#   data_dictionary = pd.read_excel(path, sheet_name = sheet_num)
#   if any([col.startswith('Unnamed:') for col in data_dictionary.columns]):
#     data_dictionary = pd.read_excel(path, sheet_name = sheet_num, header = 1)
#   data_dictionary['data_uuid'] = data_uuid
#   return data_dictionary

# metadata_df['data_dictionary'] = metadata_df[['data_dictionary_file_path','sheet1_is_data_dictionary','sheet2_is_data_dictionary','data_uuid']].progress_apply(lambda row: get_data_dictionary(row['data_dictionary_file_path'], (1 if row['sheet1_is_data_dictionary'] else 2), row['data_uuid']) if isinstance(row['data_dictionary_file_path'],str) else np.nan,axis=1)

# metadata_df[['data_uuid','excel_attachments',
#        'header_and_first_row_of_excel_files_sheet1',
#        'excel_header_signature_sheet1', 'attachment_is_data_dictionary_sheet1',
#        'header_and_first_row_of_excel_files_sheet2',
#        'excel_header_signature_sheet2', 'attachment_is_data_dictionary_sheet2',
#        'data_dictionary_file_indices_in_excel_attachments',
#        'data_dictionary_file_path', 'sheet1_is_data_dictionary',
#        'sheet2_is_data_dictionary', 'data_dictionary']].to_csv('/content/drive/MyDrive/metavis/temp.csv',index=False)

In [ ]:
# data_dictionary_paths = sorted(glob('/content/drive/MyDrive/metavis/data_dictionaries/*'))

# data_dictionary_list = []

# for p in tqdm(data_dictionary_paths):
#   tdf = pd.read_csv(p)
#   tdf.columns = [re.sub(r'\(.*?\)','',col).strip() for col in tdf.columns]
#   col_name_correction_mapping = {}
#   for col in tdf.columns:
#     if 'notes' in col.lower():
#       col_name_correction_mapping[col] = 'notes'
#       continue
#     if 'table' in col.lower() or 'unnamed' in col.lower():
#       col_name_correction_mapping[col] = 'REMOVE'
#       continue
#     if 'term' in col.lower() or 'expected' in col.lower():
#       col_name_correction_mapping[col] = 'term'
#       continue
#     if 'description' in col.lower():
#       col_name_correction_mapping[col] = 'description'
#       continue
#     if 'name' in col.lower():
#       col_name_correction_mapping[col] = 'name'
#       continue
#     if 'type' in col.lower():
#       col_name_correction_mapping[col] = 'data_type'
#       continue
#     if 'data_uuid' == col:
#       col_name_correction_mapping[col] = 'data_uuid'
#       continue
#     col_name_correction_mapping[col] = 'REMOVE'

#   tdf = tdf.rename(columns=col_name_correction_mapping)
  
#   if 'REMOVE' in tdf.columns:
#     tdf = tdf.drop(['REMOVE'],axis=1)
  
#   tdf = (tdf.dropna(how='all',axis=1)).T.drop_duplicates().T

#   if len(tdf.columns) != len(set(tdf.columns)):
#     tdf = tdf.T.reset_index().drop_duplicates(subset=['index'],keep='first').set_index('index').T
#     print('Duplicated column names occur')

#   data_dictionary_list.append(tdf)

# data_dictionary_df = pd.DataFrame()
# for tdf in tqdm(data_dictionary_list):
#   data_dictionary_df = data_dictionary_df.append(tdf, ignore_index = True)

# data_dictionary_df = data_dictionary_df[['data_uuid','name','description','data_type','term','notes']]
# data_dictionary_df.to_csv(X'/content/drive/MyDrive/metavis/combined_data_dictionary.csv',index=False)

### Transform the column information within metadata file

In [ ]:
metadata_df = metadata_df[['data_uuid','columns']].copy()
col_metadata_df = metadata_df.explode('columns').reset_index(drop=True)
col_metadata_df = col_metadata_df.dropna(subset=['columns']).reset_index(drop=True)
col_metadata_df = pd.concat([col_metadata_df['data_uuid'], pd.json_normalize(col_metadata_df['columns'])], axis=1)
# col_metadata_df.notnull().mean().sort_values(ascending=False)[:7].index.tolist()
col_metadata_df = col_metadata_df[['data_uuid', 'id', 'tableColumnId', 'name', 'fieldName', 'dataTypeName', 'description']]
col_metadata_df['description'] = col_metadata_df['description'].replace('',np.nan)

In [ ]:
import gensim
import gensim.downloader as api
# glove_model = api.load('glove-wiki-gigaword-100') # glove-100dimension is 128 MB

def remove_domain_frequent_words(input, domain_frequent_words = {}): # manually picked from 100 most common tokens
  output = list(set(input).difference(domain_frequent_words))
  return output

def get_nan_embedding(wv_model):
  na_array = np.empty((wv_model.vector_size,))
  na_array[:] = np.nan
  return na_array

def get_embedding(token, wv_model):
  try:
    return wv_model.wv[token]
  except:
    return get_nan_embedding(wv_model)

def create_embedding(data, col, domain_frequent_words = {}):
  data['_tokens'] = data[col].fillna('').apply(gensim.parsing.preprocessing.remove_stopwords).apply(gensim.utils.simple_preprocess)
  data['_tokens_wo_domain_freq'] = data['_tokens'].apply(lambda li: remove_domain_frequent_words(li, domain_frequent_words = domain_frequent_words))
  data[col+'__glove_word_embedding'] = data['_tokens_wo_domain_freq'].apply(lambda li: np.nanmean([get_embedding(x, wv_model = glove_model) for x in li], axis=0) if len(li)>0 else get_nan_embedding(wv_model = glove_model) )
  data = data.drop(['_tokens','_tokens_wo_domain_freq'],axis=1)
  return data


In [ ]:
col_metadata_df['name'] = col_metadata_df['name'].apply(lambda x: x.replace('_',' '))
col_metadata_df = create_embedding(col_metadata_df, 'name', {'nyc', 'data'})
col_metadata_df = create_embedding(col_metadata_df, 'description', {'nyc', 'data'})
col_name_based_word_embedding_for_datasets = col_metadata_df.groupby('data_uuid')['name__glove_word_embedding'].apply(list).apply(lambda li: np.nanmean(li, axis=0)).reset_index().rename(columns = {'name__glove_word_embedding':'colnames__glove_word_embedding'})
col_name_based_word_embedding_for_datasets['colnames__glove_word_embedding'] = col_name_based_word_embedding_for_datasets['colnames__glove_word_embedding'].apply(lambda arr: np.nan if all(np.isnan(arr)) else arr)

In [ ]:
name_and_description_df = metadata_df[['data_uuid','name','description']].copy()
name_and_description_df = create_embedding(name_and_description_df, 'name', {'nyc', 'data'})
name_and_description_df = create_embedding(name_and_description_df, 'description', {'nyc', 'data'})
name_and_description_df = name_and_description_df.rename(columns = {'name__glove_word_embedding':'dataset_name__glove_word_embedding'})
name_and_description_df = name_and_description_df.rename(columns = {'description__glove_word_embedding':'dataset_desc__glove_word_embedding'})

name_and_description_df['dataset_name__glove_word_embedding'] = name_and_description_df['dataset_name__glove_word_embedding'].apply(lambda arr: np.nan if all(np.isnan(arr)) else arr)
name_and_description_df['dataset_desc__glove_word_embedding'] = name_and_description_df['dataset_desc__glove_word_embedding'].apply(lambda arr: np.nan if all(np.isnan(arr)) else arr)

In [ ]:
dataset_embedding_df = pd.merge(name_and_description_df,  col_name_based_word_embedding_for_datasets, how = 'left')

In [ ]:
dataset_embedding_df['dataset_embedding'] = dataset_embedding_df[['colnames__glove_word_embedding','dataset_name__glove_word_embedding','dataset_desc__glove_word_embedding']].apply(lambda row: row['colnames__glove_word_embedding'] if not isinstance(row['colnames__glove_word_embedding'],float) else row['dataset_name__glove_word_embedding'] if not isinstance(row['dataset_name__glove_word_embedding'], float) else row['dataset_desc__glove_word_embedding'] ,axis=1)

In [ ]:
assert(dataset_embedding_df['dataset_embedding'].isnull().mean()==0)

In [ ]:
dataset_embedding_df[['data_uuid','dataset_embedding']].to_csv('/content/drive/MyDrive/metavis/dataset_embedding_v20211117.csv',index=False)

In [ ]:
import ast
import numpy as np
def from_np_array(array_string):
  # https://stackoverflow.com/a/42756309
  array_string = ','.join(array_string.replace('[ ', '[').split())
  return np.array(ast.literal_eval(array_string))
test = pd.read_csv('/content/drive/MyDrive/metavis/dataset_embedding_v20211117.csv', converters={'dataset_embedding': from_np_array})

In [ ]:
test

,data_uuid,dataset_embedding
0,2232-dj5q,"[0.272886902, 0.651672661, 0.164423525, 0.084857516, -0.0819666684, 0.453292221, 0.0757249966, 0.303720921, -0.2717686, 0.226315737, 0.287404418, -0.0704451799, 0.387170464, 0.00168698095, 0.26273..."
1,22gm-5ceg,"[-0.03767802, -0.07734199, -0.1406482, -0.09372941, 0.0608364, -0.04342001, -0.403372, 0.51750602, 0.39915959, 0.325266, -0.40449339, -0.25346, 0.16364799, -0.38072072, 0.03767188, -0.1352002, 0.3..."
2,22rr-ujq3,"[-0.09199004, -0.09967875, 0.00158188, -0.05729731, -0.36092069, 0.00770345, 0.01021996, 0.27899932, 0.39187778, 0.40246545, -0.06321181, -0.14092162, 0.2092265, 0.33034902, 0.11701665, -0.2119052..."
3,23z9-6uk9,"[0.11498749, 0.33656264, 0.27530958, 0.07165801, -0.20154372, -0.10274882, -0.23348022, 0.39350633, -0.05822064, 0.52232129, 0.12846081, -0.09404669, -0.08438362, 0.04706557, 0.07147856, -0.404043..."
4,242c-ru4i,"[-0.145268378, 0.0265072141, 0.456168638, -0.2467472, -0.0994892914, 0.46503904, -0.188583733, 0.425135664, 0.157950454, 0.287180073, -0.276847809, -0.446594164, 0.0963694876, 0.0182594338, 0.1230..."
...,...,...
3400,zmut-au2w,"[-0.01288117, 0.07701358, 0.34927, -0.17704557, 0.05406221, 0.1329609, -0.13549282, 0.558011, 0.12050997, 0.3571985, -0.01382237, -0.44943655, 0.39915407, -0.2620855, 0.21960932, -0.1710816, 0.422..."
3401,zpd4-gad8,"[0.24290453, 0.2938355, 0.5160438, 0.48103797, -0.33981597, -0.4523803, 0.087005, 0.16123514, -0.3518411, 0.60383534, 0.3476553, -0.40914786, 0.2009807, -0.05200132, 0.32497835, -0.02049678, -0.16..."
3402,zs4w-c9cd,"[0.23196084, 0.32420789, 0.49991611, 0.53018386, -0.414813, -0.52255234, 0.11655534, 0.1053189, -0.38745515, 0.6010738, 0.41343092, -0.34503776, 0.1917534, -0.04941021, 0.26384363, 0.03953277, -0...."
3403,zt9s-n5aj,"[0.06587534, 0.397745, 0.19091167, 0.26245047, -0.19906493, 0.16047067, -0.18598931, 0.01881167, -0.20354566, 0.31177306, 0.01801765, -0.27548981, 0.134378, 0.11744719, -0.01950953, -0.16392793, 0..."
